In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
# coding: utf-8
import sys
sys.path.append('/content/gdrive/MyDrive/AI/Deep-Learning/밑바닥부터 시작하는 딥러닝2/예제')
import numpy as np
from common.layers import MatMul

In [7]:
# 샘플 맥락 데이터
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# 계층 생성
in_layer0 = MatMul(W_in) ##입력층
in_layer1 = MatMul(W_in) ## 은닉층
out_layer = MatMul(W_out) ##출력층

# 순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)
print(s)


[[ 1.40918232  0.18784312 -1.54812252  0.15065431 -1.39899821  0.01931281
   0.56470077]]
